# Normal Day Analysis

In [35]:
import pandas as pd
import numpy as np
import ipaddress
import dns.resolver
import dns.reversename
import pygeoip
import matplotlib.pyplot as plt 

In [36]:
datafile='test7.parquet'

In [37]:
gi=pygeoip.GeoIP('./GeoIP.dat')
gi2=pygeoip.GeoIP('./GeoIPASNum.dat')
addr='193.136.73.21'
cc=gi.country_code_by_addr(addr)
org=gi2.org_by_addr(addr)
print(cc,org)

PT AS1930 Fundacao para a Ciencia e a Tecnologia, I.P.


In [38]:
data=pd.read_parquet(datafile)

In [39]:
data.head()

,timestamp,src_ip,dst_ip,proto,port,up_bytes,down_bytes
index,,,,,,,
781742,1550453,192.168.107.134,192.168.107.227,udp,53,199,474
781743,1550545,192.168.107.134,142.250.184.174,tcp,443,10327,59149
781744,1550546,192.168.107.134,142.250.184.174,tcp,443,9284,58110
781745,1550616,192.168.107.134,142.250.184.174,tcp,443,7955,71231
781746,1550760,192.168.107.134,142.250.184.174,tcp,443,6106,79662


In [40]:
NET=ipaddress.IPv4Network('192.168.107.0/24')
private_dst_normal = data[data.apply(lambda x: ipaddress.IPv4Address(x['dst_ip']) in NET,axis=1)]

private_dst_normal.head()


,timestamp,src_ip,dst_ip,proto,port,up_bytes,down_bytes
index,,,,,,,
781742,1550453,192.168.107.134,192.168.107.227,udp,53,199,474
781761,1555264,192.168.107.134,192.168.107.234,udp,53,190,389
781762,1555268,192.168.107.134,192.168.107.234,udp,53,194,388
781763,1555274,192.168.107.134,192.168.107.234,udp,53,169,395
781767,1560348,192.168.107.134,192.168.107.234,udp,53,121,317


In [41]:
grp_private_normal = private_dst_normal.groupby(["proto","port"])['dst_ip'].apply(list)

In [42]:

https_servers = set(grp_private_normal[('tcp', 443)])
print()

dns_servers =  set(grp_private_normal[('udp', 53)])
print()

## Servers 
tcp 443 {'192.168.107.223', '192.168.107.238', '192.168.107.222', '192.168.107.226'} https

udp 53 {'192.168.107.227', '192.168.107.234'} dns


In [43]:
grouped = data.groupby(['src_ip','dst_ip']).sum()

# Calculate the ratio
grouped['ratio'] = grouped['up_bytes'] / grouped['down_bytes']
grouped
# Reset the index to make dst_ip a column again
# grouped = grouped.reset_index()

timestamp  \
src_ip          dst_ip                       
192.168.107.100 104.16.249.130   141488093   
                104.22.48.100     72663611   
                104.244.42.129   819390937   
                104.244.47.87     26913323   
                104.26.13.207     67074464   
...                                    ...   
192.168.107.99  88.157.217.145  1072668469   
                88.157.217.146   350271422   
                88.157.217.148    56185240   
                88.221.64.44     135698478   
                97.107.133.118    25547903   

                                                                            proto  \
src_ip          dst_ip                                                              
192.168.107.100 104.16.249.130  tcptcptcptcptcptcptcptcptcptcptcptcptcptcptcpt...   
                104.22.48.100   tcptcptcptcptcptcptcptcptcptcptcptcptcptcptcpt...   
                104.244.42.129  tcptcptcptcptcptcptcptcptcptcptcptcptcptcptcpt...   
                104.244.47.87                         tcptcptcptcptcptcptcptcptcp   
                104.26.13.207   tcptcptcptcptcptcptcptcptcptcptcptcptcptcptcpt...   
...                                                                           ...   
192.168.107.99  88.157.217.145  tcptcptcptcptcptcptcptcptcptcptcptcptcptcptcpt...   
                88.157.217.146  tcptcptcptcptcptcptcptcptcptcptcptcptcptcptcpt...   
                88.157.217.148                           tcptcptcptcptcptcptcptcp   
                88.221.64.44    tcptcptcptcptcptcptcptcptcptcptcptcptcptcptcpt...   
                97.107.133.118                                 tcptcptcptcptcptcp   

                                 port  up_bytes  down_bytes     ratio  
src_ip          dst_ip                                                 
192.168.107.100 104.16.249.130  18606    416652     5329752  0.078175  
                104.22.48.100   11518    289050     2517956  0.114795  
                104.244.42.129  97460   2471399    23994481  0.102999  
                104.244.47.87    3987    126016     1111322  0.113393  
                104.26.13.207    7974    208827     3153790  0.066215  
...                               ...       ...         ...       ...  
192.168.107.99  88.157.217.145  81955   2121021    19832417  0.106947  
                88.157.217.146  23922    658427     5427966  0.121303  
                88.157.217.148   3544     81995      749854  0.109348  
                88.221.64.44     9303    220648     1845226  0.119578  
                97.107.133.118   2658     56156      476165  0.117934  

[26373 rows x 6 columns]

In [44]:
mean_ratios = grouped.groupby('src_ip')['ratio'].mean().reset_index()

In [45]:
server_traffic = data[data.apply(lambda x: x['dst_ip'] in dns_servers or x['dst_ip'] in https_servers,axis=1)]


In [46]:
server_traffic.head()

,timestamp,src_ip,dst_ip,proto,port,up_bytes,down_bytes
index,,,,,,,
781742,1550453,192.168.107.134,192.168.107.227,udp,53,199,474
781761,1555264,192.168.107.134,192.168.107.234,udp,53,190,389
781762,1555268,192.168.107.134,192.168.107.234,udp,53,194,388
781763,1555274,192.168.107.134,192.168.107.234,udp,53,169,395
781767,1560348,192.168.107.134,192.168.107.234,udp,53,121,317


In [47]:
servers_grouped = server_traffic.groupby(['dst_ip','src_ip']).sum()

# Calculate the ratio
servers_grouped['ratio'] = servers_grouped['up_bytes'] / servers_grouped['down_bytes']
servers_grouped
# Reset the index to make dst_ip a column again
# grouped = grouped.reset_index()
servers_mean_ratios = servers_grouped.groupby('dst_ip')['ratio'].mean().reset_index()

In [51]:
servers_mean_ratios.sort_values(by="ratio",ascending=False)


,dst_ip,ratio
2,192.168.107.205,1.023619
0,192.168.107.114,0.998404
1,192.168.107.126,0.995649
9,192.168.107.58,0.965760
6,192.168.107.227,0.437545
7,192.168.107.234,0.436713
8,192.168.107.238,0.111088
4,192.168.107.223,0.109747
3,192.168.107.222,0.109633
5,192.168.107.226,0.109025


## ip com aumento anomalo de trafico para os servidores da empresa
| dst_ip | ratio |
|------|-------|
| 192.168.107.205 | 1.023619 |
| 192.168.107.114 | 0.998404 |
| 192.168.107.126 | 0.995649 |
| 192.168.107.58  | 0.965760 |


In [ ]:
servers_mean_ratios.to_csv("dst_ip_test.csv")
